In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Lectura del dataset
df = pd.read_csv('../data/teleCust1000t.csv')

df.head()

In [ ]:
# Identificación de variables categóricas y numéricas
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
# Mostrar información básica
print("Columnas categóricas:")
for col in categorical_columns: # type: ignore
    print(f"{col}: {df[col].nunique()} valores únicos")

print("\nColumnas numéricas:")
for col in numerical_columns: # type: ignore
    print(f"{col}: rango [{df[col].min()}, {df[col].max()}]")

In [ ]:
# Información detallada del DataFrame
print("\nInformación del DataFrame:")
print(df.info())

In [ ]:
# Estadísticas descriptivas básicas
print("\nEstadísticas descriptivas:")
print(df.describe())

# Analisis de Distribución de las variables

In [ ]:
# Configuración básica de visualización
plt.style.use('default')
sns.set_theme(style="whitegrid")

# Variables numéricas a analizar
numerical_vars = ['tenure', 'age', 'income', 'employ', 'address', 'reside']

# Crear figura para histogramas y boxplots
fig, axes = plt.subplots(len(numerical_vars), 2, figsize=(15, 24))
fig.suptitle('Distribución de Variables Numéricas', fontsize=16, y=0.95)

# Función para calcular estadísticas de asimetría y curtosis
def get_distribution_stats(data):
    return pd.Series({
        'Media': np.mean(data),
        'Mediana': np.median(data),
        'Desv. Est.': np.std(data),
        'Asimetría': stats.skew(data),
        'Curtosis': stats.kurtosis(data)
    })

# Diccionario para almacenar estadísticas
stats_dict = {}

# Crear histogramas y boxplots
for idx, var in enumerate(numerical_vars):
    # Histograma
    sns.histplot(data=df, x=var, kde=True, ax=axes[idx, 0])
    axes[idx, 0].set_title(f'Histograma de {var}')
    axes[idx, 0].set_xlabel(var)
    axes[idx, 0].set_ylabel('Frecuencia')

    # Boxplot
    sns.boxplot(data=df, y=var, ax=axes[idx, 1])
    axes[idx, 1].set_title(f'Boxplot de {var}')

    # Calcular estadísticas
    stats_dict[var] = get_distribution_stats(df[var])

# Ajustar el layout
plt.tight_layout()

# Mostrar las gráficas
plt.show()

# Crear tabla de estadísticas
stats_df = pd.DataFrame(stats_dict)
print("\nEstadísticas de distribución:")
print(stats_df.round(2))

# Calcular y mostrar percentiles
percentiles = df[numerical_vars].quantile([0.25, 0.5, 0.75])
print("\nPercentiles principales:")
print(percentiles.round(2))

# Test de normalidad
print("\nTest de Normalidad (Shapiro-Wilk):")
for var in numerical_vars:
    stat, p_value = stats.shapiro(df[var])
    print(f"{var}: p-value = {p_value:.2e}")

### Transformaciones de Variables Numéricas

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Crear una copia del dataframe para las transformaciones
df_transformed = df.copy()

# 1. Transformación logarítmica para income (debido a su alta asimetría)
df_transformed['income_log'] = np.log1p(df['income'])  # log1p para manejar valores 0

# 2. Aplicar RobustScaler a las variables con outliers
robust_scaler = RobustScaler()
variables_to_robust_scale = ['employ', 'address']
df_transformed[['employ_scaled', 'address_scaled']] = robust_scaler.fit_transform(df[variables_to_robust_scale])

# 3. Aplicar StandardScaler a variables más normales
standard_scaler = StandardScaler()
variables_to_standard_scale = ['tenure', 'age', 'reside']
df_transformed[[f'{col}_scaled' for col in variables_to_standard_scale]] = standard_scaler.fit_transform(df[variables_to_standard_scale])

# Configuración de la visualización
plt.style.use('default')
sns.set_theme(style="whitegrid")

# Crear visualización comparativa
fig, axes = plt.subplots(3, 2, figsize=(15, 20))
fig.suptitle('Comparación de Distribuciones: Original vs Transformada', fontsize=16)

# 1. Comparar distribución de income
sns.histplot(data=df, x='income', kde=True, ax=axes[0,0])
axes[0,0].set_title('Income Original')
axes[0,0].set_xlabel('Income')

sns.histplot(data=df_transformed, x='income_log', kde=True, ax=axes[0,1])
axes[0,1].set_title('Income (Log Transformada)')
axes[0,1].set_xlabel('Log Income')

# 2. Comparar variables con RobustScaler
sns.boxplot(data=df[variables_to_robust_scale], ax=axes[1,0])
axes[1,0].set_title('Variables Originales (Employ, Address)')

sns.boxplot(data=df_transformed[['employ_scaled', 'address_scaled']], ax=axes[1,1])
axes[1,1].set_title('Variables con RobustScaler')

# 3. Comparar variables con StandardScaler
sns.boxplot(data=df[variables_to_standard_scale], ax=axes[2,0])
axes[2,0].set_title('Variables Originales (Tenure, Age, Reside)')

sns.boxplot(data=df_transformed[[f'{col}_scaled' for col in variables_to_standard_scale]], ax=axes[2,1])
axes[2,1].set_title('Variables con StandardScaler')

plt.tight_layout()
plt.show()

# Mostrar estadísticas de las variables transformadas
print("\nEstadísticas de las variables transformadas:")
transformed_stats = df_transformed[[
    'income_log',
    'employ_scaled', 'address_scaled',
    'tenure_scaled', 'age_scaled', 'reside_scaled'
]].describe()
print(transformed_stats)

# Verificar normalidad de las variables transformadas
print("\nTest de Normalidad (Shapiro-Wilk) para variables transformadas:")
for column in transformed_stats.columns:
    stat, p_value = stats.shapiro(df_transformed[column].dropna())
    print(f"{column}: p-value = {p_value:.2e}")

# Guardar las correlaciones de las variables transformadas
correlation_matrix = df_transformed[[
    'income_log',
    'employ_scaled', 'address_scaled',
    'tenure_scaled', 'age_scaled', 'reside_scaled'
]].corr()

print("\nMatriz de correlaciones de variables transformadas:")
print(correlation_matrix.round(3))

### Preprocesamiento Avanzado de Datos

In [ ]:

from sklearn.preprocessing import StandardScaler, RobustScaler
import warnings
warnings.filterwarnings('ignore')

# 2. Análisis de balance de clases (ANTES del preprocesamiento)
print("\n2. ANÁLISIS DE BALANCE DE CLASES")
print("="*50)

# Calcular distribución de clases
class_distribution = df['custcat'].value_counts()
class_percentages = df['custcat'].value_counts(normalize=True) * 100

# Visualización del balance de clases
plt.figure(figsize=(12, 5))

# Subplot 1: Gráfico de barras
plt.subplot(1, 2, 1)
sns.barplot(x=class_distribution.index, y=class_distribution.values)
plt.title('Distribución de Clases (Conteo)')
plt.xlabel('Categoría de Cliente')
plt.ylabel('Número de Clientes')

# Subplot 2: Gráfico de pie
plt.subplot(1, 2, 2)
plt.pie(class_distribution, labels=class_distribution.index, autopct='%1.1f%%')
plt.title('Distribución de Clases (%)')

plt.tight_layout()
plt.show()

# Imprimir estadísticas de balance
print("\nEstadísticas de balance de clases:")
for clase, (conteo, porcentaje) in enumerate(zip(class_distribution, class_percentages), 1):
    print(f"Clase {clase}: {conteo} clientes ({porcentaje:.1f}%)")

# 3. Preprocesamiento de datos
print("\n3. PREPROCESAMIENTO DE DATOS")
print("="*50)

# Crear copia para preprocesamiento
df_processed = df.copy()

# 3.1 Transformaciones numéricas
print("\nAplicando transformaciones...")

# Transformación logarítmica para income
df_processed['income_log'] = np.log1p(df_processed['income'])

# Crear características derivadas
df_processed['income_per_year_employed'] = df_processed['income'] / (df_processed['employ'] + 1)
df_processed['stability_score'] = df_processed['address'] * df_processed['tenure'] / (df_processed['age'] - 18)

# Aplicar RobustScaler a variables con outliers
robust_scaler = RobustScaler()
robust_vars = ['employ', 'address', 'income_per_year_employed']
df_processed[['employ_scaled', 'address_scaled', 'income_year_scaled']] = \
    robust_scaler.fit_transform(df_processed[robust_vars])

# Aplicar StandardScaler a variables más normales
standard_scaler = StandardScaler()
standard_vars = ['tenure', 'age']
df_processed[['tenure_scaled', 'age_scaled']] = \
    standard_scaler.fit_transform(df_processed[standard_vars])

# Crear variables dummy para region
df_processed = pd.get_dummies(df_processed, columns=['region'], prefix='region')

# 4. Selección de características finales
print("\n4. SELECCIÓN DE CARACTERÍSTICAS Y ANÁLISIS FINAL")
print("="*50)

features_to_use = [
    'income_log', 'employ_scaled', 'address_scaled',
    'tenure_scaled', 'age_scaled', 'income_year_scaled',
    'stability_score', 'region_1', 'region_2', 'region_3'
]

# Preparar X e y
X = df_processed[features_to_use]
y = df_processed['custcat']

# 5. Análisis de correlaciones
correlation_matrix = X.corr()

# Visualización de correlaciones
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Matriz de Correlación de Variables Procesadas')
plt.tight_layout()
plt.show()

# 6. Estadísticas finales
print("\nEstadísticas de las características procesadas:")
print(X.describe().round(3))

# 7. Resumen de transformaciones
print("\nResumen de transformaciones aplicadas:")
print("1. Transformaciones logarítmicas:")
print("   - income_log")
print("\n2. Variables escaladas con RobustScaler:")
print("   - employ_scaled")
print("   - address_scaled")
print("   - income_year_scaled")
print("\n3. Variables escaladas con StandardScaler:")
print("   - tenure_scaled")
print("   - age_scaled")
print("\n4. Variables derivadas:")
print("   - income_per_year_employed")
print("   - stability_score")
print("\n5. Variables dummy:")
print("   - region_1, region_2, region_3")

# 8. Identificar correlaciones altas
print("\nCorrelaciones significativas (>0.7):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            print(f"{correlation_matrix.columns[i]} - {correlation_matrix.columns[j]}: {correlation_matrix.iloc[i, j]:.3f}")

# Preparación final para el modelo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


def prepare_final_features(df_processed, handle_multicollinearity=True):
    """
    Prepara las características finales para modelado, manejando
    multicolinealidad y valores faltantes.
    """
    # 1. Manejar el valor faltante en stability_score
    df_processed['stability_score'] = df_processed['stability_score'].fillna(
        df_processed['stability_score'].median()
    )

    # 2. Definir características base
    base_features = [
        'income_log',
        'employ_scaled',
        'age_scaled',
        'income_year_scaled',
        'region_1', 'region_2', 'region_3'
    ]

    # 3. Manejar multicolinealidad
    if handle_multicollinearity:
        # Aplicar PCA a las variables correlacionadas
        correlated_features = ['address_scaled', 'tenure_scaled', 'stability_score']
        pca = PCA(n_components=1)
        stability_component = pca.fit_transform(
            df_processed[correlated_features]
        )

        # Añadir componente PCA al dataset
        df_processed['stability_index'] = stability_component
        final_features = base_features + ['stability_index']
    else:
        # Usar todas las características
        final_features = base_features + ['address_scaled', 'tenure_scaled', 'stability_score']

    # 4. Preparar X e y
    X = df_processed[final_features]
    y = df_processed['custcat']

    # 5. Split estratificado
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.2,
        random_state=42,
        stratify=y
    )

    return X_train, X_test, y_train, y_test, final_features

# Ejemplo de uso
X_train, X_test, y_train, y_test, features = prepare_final_features(df_processed)

# Mostrar información del proceso
print("Características finales seleccionadas:")
print(features)
print("\nDimensiones de los conjuntos:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print("\nDistribución de clases en train:")
print(pd.Series(y_train).value_counts(normalize=True).round(3))
print("\nDistribución de clases en test:")
print(pd.Series(y_test).value_counts(normalize=True).round(3))

# Pipeline de Modelado

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def create_improved_pipeline():
    """
    Pipeline mejorado con más opciones y mejor preprocesamiento
    """
    # 1. Crear pipeline con preprocesamiento
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', GradientBoostingClassifier(random_state=42))
    ])

    # 2. Parámetros más exhaustivos
    param_grid = {
        'classifier__n_estimators': [100, 200, 300],
        'classifier__learning_rate': [0.01, 0.1, 0.3],
        'classifier__max_depth': [3, 5, 7],
        'classifier__min_samples_split': [2, 5],
        'classifier__subsample': [0.8, 1.0]
    }

    # 3. GridSearch con métricas más robustas
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=5,
        scoring='f1_weighted',
        n_jobs=-1,
        verbose=1
    )

    return grid_search

def evaluate_model_detailed(model, X_train, X_test, y_train, y_test, feature_names):
    """
    Evaluación más detallada del modelo
    """
    # Entrenar modelo
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # 1. Visualizaciones
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # Matriz de confusión normalizada
    conf_matrix = confusion_matrix(y_test, y_pred, normalize='true')
    sns.heatmap(conf_matrix, annot=True, fmt='.2f', cmap='YlOrRd', ax=axes[0,0])
    axes[0,0].set_title('Matriz de Confusión Normalizada')

    # Importancia de características
    if hasattr(model, 'best_estimator_'):
        importances = model.best_estimator_.named_steps['classifier'].feature_importances_
    else:
        importances = model.named_steps['classifier'].feature_importances_

    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=True)

    sns.barplot(x='importance', y='feature', data=importance_df, ax=axes[0,1])
    axes[0,1].set_title('Importancia de Características')

    # Predicciones por clase
    class_probs = model.predict_proba(X_test)
    avg_probs = np.mean(class_probs, axis=0)
    sns.barplot(x=range(len(avg_probs)), y=avg_probs, ax=axes[1,0])
    axes[1,0].set_title('Probabilidad Media por Clase')

    # Error por clase
    errors = y_test != y_pred
    error_by_class = pd.DataFrame({'error': errors, 'class': y_test})
    sns.barplot(x='class', y='error', data=error_by_class, ax=axes[1,1])
    axes[1,1].set_title('Tasa de Error por Clase')

    plt.tight_layout()
    plt.show()

    # 2. Métricas detalladas
    print("\nReporte de Clasificación Detallado:")
    print(classification_report(y_test, y_pred))

    if hasattr(model, 'best_params_'):
        print("\nMejores parámetros:")
        print(model.best_params_)

        print("\nMejor score en cross-validation:", model.best_score_)

    # 3. Análisis de confianza de predicciones
    probs = model.predict_proba(X_test)
    max_probs = np.max(probs, axis=1)

    print("\nConfianza media de predicciones:", np.mean(max_probs))
    print("Confianza mínima:", np.min(max_probs))
    print("Confianza máxima:", np.max(max_probs))

    return model

# Ejecutar el pipeline mejorado
print("Entrenando modelo mejorado...")
improved_model = create_improved_pipeline()
trained_improved_model = evaluate_model_detailed(
    improved_model,
    X_train,
    X_test,
    y_train,
    y_test,
    features
)